In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2\

import sys
import os

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from src.data.make_dataset import CRCHistoPhenotypes_Patch 

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.noise import GaussianNoise


Using TensorFlow backend.


# Load dataset

In [2]:
dataset = CRCHistoPhenotypes_Patch(patch_size=(52,52))
X, y = dataset.load_patches()

In [6]:
labelencoder = LabelEncoder()
labelencoder.fit(y)
y = labelencoder.transform(y)

In [7]:
len(X)

22444

In [8]:
batch_size = 32
num_classes = 4
epochs = 200
data_augmentation = False
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_CRCPatches_trained_model.h5'

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=42)
len_X_train = len(X_train)
X_train = np.concatenate(X_train, axis=0).reshape((len_X_train, )+X[0].shape)

len_X_test = len(X_test)
X_test = np.concatenate(X_test, axis=0).reshape((len_X_test, )+X[0].shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Define a Keras Sequential model

In [10]:
model = Sequential()
model.add(GaussianNoise(stddev=0.1, input_shape=(52, 52, 3)))
model.add(Convolution2D(filters=32, kernel_size=(7, 7), activation='relu', input_shape=(52, 52, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.1))

model.add(Convolution2D(filters=48, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.2))

model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='sigmoid'))
# model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))


In [11]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [12]:
if not data_augmentation:
    print('Not using data augmentation.')
    history = model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=True,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=True,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        fill_mode = "reflect")

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    history = model.fit_generator(datagen.flow(X_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        workers=4)

Not using data augmentation.
Train on 15037 samples, validate on 7407 samples
Epoch 1/200
15037/15037 [==============================] - 7s - loss: 1.0715 - acc: 0.5431 - val_loss: 0.8097 - val_acc: 0.6904
Epoch 2/200
15037/15037 [==============================] - 3s - loss: 0.8500 - acc: 0.6656 - val_loss: 0.7775 - val_acc: 0.7007
Epoch 3/200
15037/15037 [==============================] - 3s - loss: 0.7789 - acc: 0.6983 - val_loss: 0.8472 - val_acc: 0.6588
Epoch 4/200
15037/15037 [==============================] - 3s - loss: 0.7347 - acc: 0.7185 - val_loss: 0.7314 - val_acc: 0.7153
Epoch 5/200
15037/15037 [==============================] - 3s - loss: 0.6890 - acc: 0.7359 - val_loss: 0.7581 - val_acc: 0.7073
Epoch 6/200
15037/15037 [==============================] - 4s - loss: 0.6536 - acc: 0.7496 - val_loss: 0.8569 - val_acc: 0.6748
Epoch 7/200
15037/15037 [==============================] - 3s - loss: 0.6232 - acc: 0.7599 - val_loss: 0.6441 - val_acc: 0.7531
Epoch 8/200
15037/15037 [=

15037/15037 [==============================] - 3s - loss: 0.0584 - acc: 0.9807 - val_loss: 1.8201 - val_acc: 0.7338
Epoch 64/200
15037/15037 [==============================] - 3s - loss: 0.0482 - acc: 0.9844 - val_loss: 1.7042 - val_acc: 0.7209
Epoch 65/200
15037/15037 [==============================] - 3s - loss: 0.0456 - acc: 0.9854 - val_loss: 1.7117 - val_acc: 0.7294
Epoch 66/200
15037/15037 [==============================] - 3s - loss: 0.0488 - acc: 0.9834 - val_loss: 1.7748 - val_acc: 0.7242
Epoch 67/200
15037/15037 [==============================] - 3s - loss: 0.0576 - acc: 0.9816 - val_loss: 1.7240 - val_acc: 0.7239
Epoch 68/200
15037/15037 [==============================] - 3s - loss: 0.0461 - acc: 0.9846 - val_loss: 1.7276 - val_acc: 0.7299
Epoch 69/200
15037/15037 [==============================] - 3s - loss: 0.0539 - acc: 0.9836 - val_loss: 1.6550 - val_acc: 0.7308
Epoch 70/200
15037/15037 [==============================] - 3s - loss: 0.0462 - acc: 0.9840 - val_loss: 1.8221

15037/15037 [==============================] - 3s - loss: 0.0967 - acc: 0.9669 - val_loss: 1.5162 - val_acc: 0.7200
Epoch 127/200
15037/15037 [==============================] - 3s - loss: 0.0911 - acc: 0.9691 - val_loss: 1.5652 - val_acc: 0.7153
Epoch 128/200
15037/15037 [==============================] - 3s - loss: 0.0825 - acc: 0.9712 - val_loss: 1.5345 - val_acc: 0.7161
Epoch 129/200
15037/15037 [==============================] - 3s - loss: 0.0893 - acc: 0.9696 - val_loss: 1.6266 - val_acc: 0.7309
Epoch 130/200
15037/15037 [==============================] - 3s - loss: 0.1042 - acc: 0.9663 - val_loss: 1.5199 - val_acc: 0.7196
Epoch 131/200
15037/15037 [==============================] - 3s - loss: 0.0863 - acc: 0.9703 - val_loss: 1.7204 - val_acc: 0.7026
Epoch 132/200
15037/15037 [==============================] - 3s - loss: 0.0834 - acc: 0.9702 - val_loss: 1.7965 - val_acc: 0.7077
Epoch 133/200
15037/15037 [==============================] - 3s - loss: 0.0933 - acc: 0.9679 - val_loss:

KeyboardInterrupt: 

In [13]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [14]:
score = model.evaluate(X_test, y_test, verbose=0)
score

[1.6718768153223444, 0.72877008249132968]